<a href="https://colab.research.google.com/github/Subhash-K45/python/blob/main/Web_Scarping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import chromedriver_autoinstaller
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import requests
from tqdm import tqdm
chromedriver_autoinstaller.install()
driver = webdriver.Chrome()

def call_url(url):
    driver.get(url)

def scrap_web(url):
    call_url(url)

def return_soup():
    soup = BeautifulSoup(driver.page_source,'html.parser')
    return soup

def get_data():
    cases_title=[]
    cases_desc=[]
    acts=[]
    judges=[]
    call_url('https://www.scconline.com/blog/post/category/casebriefs/supremecourt/')
    scrap_web('https://www.scconline.com/blog/post/category/casebriefs/supremecourt/')
    original_soup = return_soup()
    for title,time in tqdm(zip(original_soup.find_all('h2',class_='entry-title'),original_soup.find_all('time'))):
        links = title.find('a').get('href')
        call_url(links)
        scrap_web(links)
        sub_soup = return_soup()
        cases_title.append(sub_soup.find('h1',class_='entry-title').text)
        for cases in sub_soup.find_all('div',class_='hfeed site wrap'):
            for case in cases.find_all('p'):
                if (case.text.startswith('[') and case.text.endswith(']')):
                    cases_desc.append(case.text)
            c=0
            try:
                for judge in sub_soup.find_all('strong'):
                    judges.append(judge.text)
                    c=1
            except Exception as e:
                print('NA')
                print(f'An error occurred: {e}')
            if c==0:
                judges.append('NA')
        for div in sub_soup.find_all('div',class_='entry-content'):
            for p in div.find_all('p'):
                for a in p.find_all('a'):
                    if 'Click' in a.text or a.text=='Learn how your comment data is processed' or len(a.text)>28:
                        continue
                    print(a.text)
                    acts.append(a.text)
        acts.append('*****')
    return cases_title,cases_desc,acts,judges

cases_title,cases_desc,acts,judges = get_data()

df = pd.DataFrame([cases_title]).T
df.rename(columns={0:'case name'})

cases_new=[]
for x in cases_desc:
    for y in (x.split(' ')):
        if 'No' in y:
            cases_new.append(y)
            cases_new.append(x.split(' ')[x.split(' ').index(y)+1])

acts_new = []
l=[]
for act in acts:
    if act=='*****' and len(l)>0:
        acts_new.append(l)
        l=[]
    else:
        if '*' not in act:
            l.append(act)

df = pd.DataFrame([['Supreme Court']*20,cases_title,judges,acts_new,cases_new])
df.T.rename(columns={0:'Court',1:'Case Names',2:'Judges',3:'Acts',4:'Doc Id'}).to_csv('Assesment.csv',index=False)